In [178]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction,isnan, when, count, col, isnull,month, hour,year,minute,second,lower,lit,dayofweek,to_timestamp,to_date
from pyspark.sql.types import TimestampType, DateType,DoubleType,FloatType
from pyspark.sql import DataFrame
from functools import reduce
import pandas as pd
import dateparser
import json
from functools import reduce
from pyspark.sql import DataFrame

In [2]:
spark = (
    SparkSession.builder
    .config('spark.executor.memory', '40g')
    .config('spark.executor.cores', '20')
    .config("spark.sql.session.timeZone", "America/Chicago")
    .config('spark.driver.memory','20g').master("spark://127.0.0.1:7077").appName("nfd")
    .getOrCreate()
)

In [242]:
path = "dump_July_2020_to_august_26_2021_no_null.csv"
incidents = spark.read.option("header",True).csv(path)

In [243]:
incidents=incidents.drop('location.type').drop('alarm_year').drop('_id').drop('respondingVehicles').drop('fireZone').drop('alarm_time').drop('alarm_datetime').drop('alarm_date').drop('location.coordinates')

In [244]:
incidents=incidents.withColumn("alarm_datetime",to_timestamp("alarmDateTime")) 

In [245]:
incidents=incidents.withColumn("arrival_datetime",to_timestamp("arrivalDateTime")) 

In [246]:
incidents=incidents.withColumn("alarm_date",to_date("arrivalDateTime")) 

In [247]:
incidents=incidents.withColumn("month",month("alarm_date")) 

In [248]:
incidents=incidents.withColumn("alarm_year",year("alarm_date")) 

In [249]:
incidents=incidents.withColumn("latitude", incidents.latitude.cast('double'))

In [250]:
incidents=incidents.withColumn("longitude", incidents.longitude.cast('double'))

In [251]:
incidents=incidents.withColumn("responsetime", (incidents.arrival_datetime.cast('long')-incidents.alarm_datetime.cast('long'))/60)

In [252]:
incidents=incidents.drop('arrival_datetime')

In [253]:
oldincidents=spark.read.load('/Users/abhishek/Desktop/nfd-dashboard/app/data/nfd/incidents_july_2020_2.parquet')

In [254]:
oldincidents=oldincidents.drop('__null_dask_index__').drop('_id').drop('location.type').drop('time').drop('dayofweek').drop('respondingVehicles').drop('fireZone').drop('alarm_time').drop('location.coordinates')

In [255]:
oldincidents=oldincidents.withColumn("month",month("alarm_date")) 

In [256]:
oldincidents=oldincidents.withColumn("alarm_year",year("alarm_date")) 

In [257]:
oldincidents=oldincidents.sort(oldincidents.alarm_datetime.asc())

In [258]:
oldincidents.take(4)

[Row(incidentNumber='FFD18123100136507', latitude=36.15343989, longitude=-86.76606078, emdCardNumber='10D4', alarmDateTime='2019-01-01T00:04:01.677Z', arrivalDateTime='2018-12-31T18:06:35.78-06:00', weather='rain', alarm_datetime=datetime.datetime(2018, 12, 31, 12, 4, 1, 677000), alarm_date=datetime.datetime(2018, 12, 30, 18, 0), alarm_year=2018, responsetime=2.5683833333333337, month=12),
 Row(incidentNumber='FFD18123100136508', latitude=36.18651573, longitude=-86.77385822, emdCardNumber='26A1', alarmDateTime='2019-01-01T00:05:11.157Z', arrivalDateTime='2018-12-31T18:15:08.96-06:00', weather='rain', alarm_datetime=datetime.datetime(2018, 12, 31, 12, 5, 11, 157000), alarm_date=datetime.datetime(2018, 12, 30, 18, 0), alarm_year=2018, responsetime=9.963383333333333, month=12),
 Row(incidentNumber='FFD18123100136509', latitude=36.14180436, longitude=-86.78402494, emdCardNumber='17A2G', alarmDateTime='2019-01-01T00:12:43.227Z', arrivalDateTime='2018-12-31T18:17:49.52-06:00', weather='rain'

In [259]:
oldincidents.printSchema()

root
 |-- incidentNumber: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- emdCardNumber: string (nullable = true)
 |-- alarmDateTime: string (nullable = true)
 |-- arrivalDateTime: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- alarm_datetime: timestamp (nullable = true)
 |-- alarm_date: timestamp (nullable = true)
 |-- alarm_year: integer (nullable = true)
 |-- responsetime: double (nullable = true)
 |-- month: integer (nullable = true)



In [260]:
incidents.printSchema()

root
 |-- incidentNumber: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- emdCardNumber: string (nullable = true)
 |-- alarmDateTime: string (nullable = true)
 |-- arrivalDateTime: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- alarm_datetime: timestamp (nullable = true)
 |-- alarm_date: date (nullable = true)
 |-- month: integer (nullable = true)
 |-- alarm_year: integer (nullable = true)
 |-- responsetime: double (nullable = true)



In [261]:
allincidents = incidents.union(oldincidents.select(incidents.columns))
allincidents=allincidents.dropDuplicates(['incidentNumber']) 
allincidents=allincidents.sort(allincidents.alarm_datetime.asc())

In [262]:
allincidents.take(4)

[Row(incidentNumber='FFD18123100136507', latitude=36.15343989, longitude=-86.76606078, emdCardNumber='10D4', alarmDateTime='2019-01-01T00:04:01.677Z', arrivalDateTime='2018-12-31T18:06:35.78-06:00', weather='rain', alarm_datetime=datetime.datetime(2018, 12, 31, 12, 4, 1, 677000), alarm_date=datetime.datetime(2018, 12, 30, 18, 0), month=12, alarm_year=2018, responsetime=2.5683833333333337),
 Row(incidentNumber='FFD18123100136508', latitude=36.18651573, longitude=-86.77385822, emdCardNumber='26A1', alarmDateTime='2019-01-01T00:05:11.157Z', arrivalDateTime='2018-12-31T18:15:08.96-06:00', weather='rain', alarm_datetime=datetime.datetime(2018, 12, 31, 12, 5, 11, 157000), alarm_date=datetime.datetime(2018, 12, 30, 18, 0), month=12, alarm_year=2018, responsetime=9.963383333333333),
 Row(incidentNumber='FFD18123100136509', latitude=36.14180436, longitude=-86.78402494, emdCardNumber='17A2G', alarmDateTime='2019-01-01T00:12:43.227Z', arrivalDateTime='2018-12-31T18:17:49.52-06:00', weather='rain'

In [263]:
allincidents.write.option("mapreduce.fileoutputcommitter.algorithm.version", "2").format("parquet").save("nfdincidents_till_July2021.parquet")